<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/notebook_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Das Projekt aus dem Github klonen und in den Projektsordner**

In [1]:
#wenn die Ordner noch nicht geklont ist, soll dieser Fehler zuerst durchgeführt werden.
!git clone https://github.com/hanhluukim/replication-topic-modelling-in-embedding-space.git

Cloning into 'replication-topic-modelling-in-embedding-space'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 271 (delta 28), reused 35 (delta 17), pack-reused 225
Receiving objects: 100% (271/271), 4.48 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [2]:
cd /content/replication-topic-modelling-in-embedding-space

/content/replication-topic-modelling-in-embedding-space


#**Die benötige Paketen für das Projekt mittels requirements.txt installieren**



In [3]:
# Falls die Packages noch nicht installiert wurden, 
!pip install -r "/content/replication-topic-modelling-in-embedding-space/requirements.txt"

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 88 kB 4.6 MB/s 
     |████████████████████████████████| 28.8 MB 5.1 MB/s 
     |████████████████████████████████| 636 kB 43.1 MB/s 
     |████████████████████████████████| 79.9 MB 94 kB/s 
     |████████████████████████████████| 880 kB 40.3 MB/s 
     |████████████████████████████████| 6.6 MB 28.0 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 34.0 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=233f20d9bd1e3089dda02a68bf8ba7a40acb0f0b5315cfa0d8f20285098f3d42
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=cfc50bc9bb8fb998f19ab1bff88d13999b531e107bb6b693ffa8362d6f57d3da
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80

# **Gebrauchte Paketen importieren**

In [4]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import umap.umap_ as umap
import time
import plotly.express as px
from sklearn import cluster
from sklearn import metrics

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Vorverarbeitung und BOW-Repräsentationen für Textdaten durchführen**
1. Vocabular erstellen
2. BOW-Repräsentationen für allen Teildatensätzen

In [5]:
from src.preprare_dataset import TextDataLoader

In [6]:
# init TextDataLoader für die Datenquelle 20 News Groups
# Daten abrufen vom Sklearn, tokenisieren und besondere Charaktern entfernen
textsloader = TextDataLoader(source="20newsgroups", train_size=None, test_size=None)
textsloader.load_tokenize_texts("20newsgroups")
# Beispiel von Textdaten
textsloader.show_example_raw_texts(n_docs=2)

loading texts: ...
finished load!
check some sample texts of the dataset
['From', ':', 'lerxst', '@', 'wam', '.', 'umd', '.', 'edu', '(', "where's", 'my', 'thing', ')', 'Subject', ':', 'WHAT', 'car', 'is', 'this', '!', '?', 'Nntp', 'Posting', 'Host', ':', 'rac3', '.', 'wam', '.', 'umd', '.', 'edu', 'Organization', ':', 'University', 'of', 'Maryland', ',', 'College', 'Park', 'Lines', ':', '15', 'I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2', 'door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s', '/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs'

In [7]:
# Vorverarbeitung von Daten mit folgenden Schritten:
textsloader.preprocess_texts(length_one_remove=True, punctuation_lower = True, stopwords_filter = True)

start: preprocessing: ...
finised: preprocessing!


In [8]:
# Daten zerlegen für Train, Test und Validation. Erstellen Vocabular aus dem Trainset
textsloader.split_and_create_voca_from_trainset(max_df=0.7, min_df=10, stopwords_remove_from_voca=True)

permuted indices for the train set: [ 56  51  98 182 139  63 121  45  46  12 223 136  66 158  21]
start creating vocabulary ...
length of the vocabulary: 348
sample ten words of the vocabulary: ['coming', 'understanding', 'cost', 'windows', 'world', 'engineering', 'question', 'light', 'takes', 'months']
length word2id list: 348
length id2word list: 348
finished: creating vocabulary


In [9]:
# Erstellen BOW-Repräsentation für ETM Modell
for_lda_model = False 
word2id, id2word, train_set, test_set, val_set = textsloader.create_bow_and_savebow_for_each_set(for_lda_model=for_lda_model)

length train-documents-indices : 4470
length of the vocabulary: 348


start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 149
min doc-id: 0
all docs: 4470
all words: 4470
docidx unique 150
words unique: 348
ndocs: 150
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
max word-id: 347
min word-id: 0
max doc-id: 49
min doc-id: 0
all docs: 1465
all words: 1465
docidx unique 50
words unique: 325
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [3, 4, 5, 6, 9, 12, 13, 14, 15, 19]
max word-id: 347
min word-id: 3
max doc-id: 49
min doc-id: 0
all docs: 719
all words: 719
docidx unique 50
words unique: 240
ndocs: 50
vocab-size: 348
finised creating bow input!

start: creating bow representation...
top 10 - word-id of the doc: [0, 1, 2, 3, 4, 6

# **Vocabular und IDs anzeigen als Beispiel**

---





In [10]:
# show for samples: 100 word2id and id2 word
word2id_df_100 = pd.DataFrame()
word2id_df_100['word'] = list(word2id.keys())[:100]
word2id_df_100['id'] = list(word2id.values())[:100]
word2id_df_100

,word,id
0,coming,0
1,understanding,1
2,cost,2
3,windows,3
4,world,4
...,...,...
95,public,95
96,good,96
97,early,97
98,small,98


# **Die Größe von Datensätzen kontrollieren**

In [11]:
# Kontrollieren die Größen von verschiedenen Datensätzen
print(f'Size of the vocabulary after prprocessing ist: {len(textsloader.vocabulary)}')
print(f'Size of train set: {len(train_set["tokens"])}')
print(f'Size of val set: {len(val_set["tokens"])}')
print(f'Size of test set: {len(test_set["test"]["tokens"])}')

Size of the vocabulary after prprocessing ist: 348
Size of train set: 150
Size of val set: 100
Size of test set: 50


# **Dokumenten wiederstellen für Word2Vec Embedding**

---



In [12]:
# re-erstellen von Dokumenten nach der Vorverarbeitungen. Die Dokumenten sind in Wörtern und werden für Word-Embedding Training benutzt
docs_tr, docs_t, docs_v = textsloader.get_docs_in_words_for_each_set()
train_docs_df = pd.DataFrame()
train_docs_df['text-after-preprocessing'] = [' '.join(doc) for doc in docs_tr[:100]]
train_docs_df

,text-after-preprocessing
0,washington keywords article nntp posting host ...
1,state university religion religion christian b...
2,ac uk free distribution world department compu...
3,nasa gov related nasa research center usa nntp...
4,ca university love question bible long time qu...
...,...
95,hp reply robert message apr nntp posting host ...
96,de point view reply de center university david...
97,big nntp posting host university article write...
98,info summary hard nntp posting host university...


# **Word-Embedding trainieren mit dem Traindatensatz**

In [13]:
from src.embedding import WordEmbeddingCreator
save_path = Path.joinpath(Path.cwd(), "vocab_embedding.txt")
wb_creator = WordEmbeddingCreator(model_name="cbow", documents = docs_tr, save_path= save_path)
wb_creator.train(min_count=0, embedding_size= 10)
vocab = list(word2id.keys())
wb_creator.create_and_save_vocab_embedding(vocab, save_path)

word-embedding train begins
word-embedding train finished
length of vocabulary from word-embedding model 348
length of the vocabulary of prepraring-dataset-vocabulary: 348


100%|██████████| 348/348 [00:00<00:00, 25088.40it/s]


True

In [14]:
v = list(wb_creator.model.wv.vocab)[0]
vec = list(wb_creator.model.wv.__getitem__(v))
print(f'word-embedding of the word-- {v}: ')
print(f'vector: {vec}')
print(f'dim of vector: {len(vec)}')

word-embedding of the word-- washington: 
vector: [-0.0616175, 0.08143439, 0.055193506, -0.0573173, -0.007247365, -0.013431087, 0.07458643, -0.026739035, 0.0075050024, 0.11852459]
dim of vector: 10


# **Word-Embeddings visualieren als Beispiel**


In [15]:
# read word-embedding files
with open(save_path) as f:
  lines = f.readlines()
embedding_data = []
words_data = []
for t in lines:
  w = t.split("\t")[0]
  v = [float(e) for e in t.split("\t")[1].split(" ")]
  words_data.append(w)
  embedding_data.append(v)

In [16]:
# clustering words with KMeans and Words-Vectors
kmeans = cluster.KMeans(n_clusters=10)
kmeans.fit(embedding_data)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)

Cluster id labels for inputted data
[0 6 9 1 9 1 7 0 6 6 6 6 5 6 7 8 5 2 5 8 5 5 3 5 9 1 5 3 4 3 1 7 0 7 9 8 7
 1 4 9 5 3 7 1 1 4 6 4 9 6 1 5 0 0 4 2 5 5 5 9 2 6 3 0 1 9 6 3 8 5 7 1 5 3
 3 9 7 1 0 2 6 5 1 2 3 7 8 5 8 6 2 4 7 8 2 4 1 2 8 7 7 9 3 5 7 4 5 0 6 0 7
 8 6 4 1 0 8 0 8 1 2 8 0 2 2 2 4 4 1 3 0 3 1 2 5 2 5 4 5 6 3 4 4 8 0 0 5 3
 5 3 7 4 2 8 3 2 1 8 1 1 5 5 5 5 4 2 8 4 2 2 3 1 6 1 3 7 5 1 8 5 7 2 8 1 8
 5 7 4 0 3 1 0 4 8 8 1 7 8 3 6 1 2 3 3 7 2 2 0 4 3 2 7 4 1 3 1 4 7 1 2 3 7
 1 6 1 2 1 3 4 6 1 0 7 8 2 0 2 2 4 6 7 8 1 0 1 6 5 0 2 4 6 0 2 2 2 2 4 2 2
 1 8 0 9 6 1 6 9 1 4 7 7 4 6 3 7 2 1 6 6 4 3 8 0 7 8 2 2 2 8 3 6 0 4 6 1 2
 4 5 4 4 1 1 6 2 2 0 4 6 8 3 2 8 2 7 7 4 4 7 0 2 2 2 2 8 8 4 4 2 8 6 2 6 7
 2 2 8 0 0 0 5 2 7 3 3 1 7 2 2]
Centroids data
[[-0.0499181   0.07159634  0.00037833 -0.0186424   0.01913697 -0.03438601
   0.06415873  0.00364077  0.0387983   0.12947281]
 [-0.05388258  0.13663567  0.05956406 -0.02224701  0.05255082 -0.06071362
   0.06171536  0.0261201   0.07608133  0.25

In [17]:
# dimension reduction with umap
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(embedding_data)
print('Duration: {} seconds'.format(time.time() - start))

# show samples after dim-reduction in dataframe
wb = pd.DataFrame(embedding, columns=['x', 'y', 'z'])
wb['word'] = words_data
wb['cluster'] = ['cluster ' + str(c) for c in labels]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Duration: 17.223621129989624 seconds


In [18]:
# visualization
fig = px.scatter_3d(wb, 
                    text = wb['word'],
                    x='x', y='y', z='z',
                    color = wb['cluster'],
                    title ="word-embedding-samples")
fig.show()

# **ETM-Model trainieren**

In [19]:
# using DocSet to use easier the modul DataSet from torch
from src.train_etm import DocSet, ETMTrain
from src.etm import ETM

vocab_size = len(list(word2id.keys()))
tr_set = DocSet("train", vocab_size, train_set)
print(len(tr_set))
print(tr_set.__getitem__(0))

150
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 

In [20]:
num_topics = 5
t_hidden_size = 100
rho_size = len(embedding_data[0])
emb_size = len(embedding_data[0])
theta_act = "relu"

class TrainArguments:
      def __init__(self, epochs, batch_size, log_interval):
          self.epochs = epochs
          self.batch_size = batch_size
          self.log_interval = log_interval

class OptimizerArguments:
      def __init__(self, optimizer_name, lr, wdecay):
            self.optimizer = optimizer_name
            self.lr = lr
            self.wdecay = wdecay
            
train_args = TrainArguments(epochs=10, batch_size=100, log_interval=None)
optimizer_args = OptimizerArguments(optimizer_name="Adam", lr=0.005, wdecay=0.1)

print(train_args.epochs)
print(optimizer_args.optimizer)

training_set = train_set

# define the ETM-model with setting-parameters
etm_model = ETM(
      num_topics, 
      vocab_size, 
      t_hidden_size, rho_size, emb_size, theta_act, 
      embedding_data, enc_drop=0.5)

# start training
train_class = ETMTrain().train(
    etm_model,
    num_topics, 
    vocab_size, 
    train_args, optimizer_args, training_set, 
    t_hidden_size, rho_size, emb_size, theta_act, embedding_data, 0.5)

10
Adam
Defaulting to vanilla SGD
Epoch 0/10:
torch.Size([100])
torch.Size([])
total loss: 16049.3115234375
Epoch 1/10:
torch.Size([100])
torch.Size([])
total loss: 18193.31640625
Epoch 2/10:
torch.Size([100])
torch.Size([])
total loss: 18749.890625
Epoch 3/10:
torch.Size([100])
torch.Size([])
total loss: 18192.6796875
Epoch 4/10:
torch.Size([100])
torch.Size([])
total loss: 16852.69921875
Epoch 5/10:
torch.Size([100])
torch.Size([])
total loss: 18099.162109375
Epoch 6/10:
torch.Size([100])
torch.Size([])
total loss: 17802.39453125
Epoch 7/10:
torch.Size([100])
torch.Size([])
total loss: 16653.046875
Epoch 8/10:
torch.Size([100])
torch.Size([])
total loss: 17403.05859375
Epoch 9/10:
torch.Size([100])
torch.Size([])
total loss: 17507.982421875
